# CSI 4142 Deliverable 2 - Phase 2
### Group 35
- Yasin Elmi, 300163765
- Oluwatobiloba Ogunbi, 300202843
- Michael Thompson, 300175414

## Intial reading of files into dataframes

Note: may need to adjust the file name and/or location if this notebook is not within the same directory as the data files.

In [192]:
import pandas as pd
import json

df_movies = pd.read_csv('datasets/tmdb_5000_movies.csv', index_col=False)
df_credits = pd.read_csv('datasets/tmdb_5000_movies.csv')

# Transforming of data
The following section is the process of our data being cleaned and transformed

## Dropping columns
Initial dropping of unnecessary columns, according to our first deliverable plan

In [193]:
# Dropping unneccessary columns
df_movies = df_movies.drop('homepage', axis=1)
#df_movies = df_movies.drop('id', axis=1)
df_movies = df_movies.drop('keywords', axis=1)
df_movies = df_movies.drop('overview', axis=1)
df_movies = df_movies.drop('status', axis=1)
df_movies = df_movies.drop('tagline', axis=1)

## Dropping rows
We will begin by initially dropping rows where both the 'budget' and 'revenue' value are 0 (previously determined that there exists no null values in these columns). Both of these values are pertinent to our investigation, and it would be inappropriate to assign an estimated or calculated value for both of these columns. Thus, in the case of our investigatin, we will drop these rows (890 rows).

In [194]:
# Dropping all rows where both the budget and revenue column are 0
# These values being 0 present no value to our investigation, and would be filled with 
print(len(df_movies))
index_revenue_budget = df_movies[ (df_movies['revenue']==0) & (df_movies['budget']==0) ].index
df_movies = df_movies.drop(index_revenue_budget)
print(len(df_movies))


4803
3913


### Checking for null
Below we check for rows that are null. In our case, there are only two null values that are present found in the 'runtime' column. Since it isn't appropriate to assume or calculate a potential runtime based on other movies, we will assign all movies with runtime 'Nan' or '0' as -1. Thus, during our investigations we are aware of the incomplete data present for consideration.

In [195]:
# Check for other Null or NaN values
df_movies[df_movies.isnull().any(axis=1)]

,budget,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count
2656,15000000,"[{""id"": 18, ""name"": ""Drama""}]",370980,it,Chiamatemi Francesco - Il Papa della gente,0.738646,"[{""name"": ""Taodue Film"", ""id"": 45724}]","[{""iso_3166_1"": ""IT"", ""name"": ""Italy""}]",2015-12-03,0,NaN,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Chiamatemi Francesco - Il Papa della gente,7.3,12
4140,2,"[{""id"": 99, ""name"": ""Documentary""}]",459488,en,"To Be Frank, Sinatra at 100",0.050625,"[{""name"": ""Eyeline Entertainment"", ""id"": 60343}]","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2015-12-12,0,NaN,[],"To Be Frank, Sinatra at 100",0.0,0


### Checking for 'Noisy' values
Below is an investigation on 'noisy' values, which appear to be empty cells assigned some place holder value. They are as follows
- 147 rows with df_movies['budget'] = 0
- 4 rows with df_movies['genres'] = '[]' (placeholder for an empty cell)
- 0 rows with df_movies['original_language'] = ''
- 0 rows with df_movies['original_title'] = ''
- 0 rows with df_movies['popularity'] = 0
- 100 rows with df_movies['production_companies'] = '[]' (placeholder for an empty cell)
- 47 rows with df_movies['production_countries'] = '[]' (placeholder for an empty cell)
- 537 rows with df_movies['revenue'] = 0
- 0 rows with df_movies['title'] = ''
- 3 rows with df_movies['runtime'] = 0 + 2 rows with df_movies['runtime'] = NaN (as detailed above)
- 166 rows where the df_movies['original_title'] != df_movies['title']. This can be assumed that foreign films have an original title name different than it's English title. This is useful to know, as when staging we will utilize the 'title' column and not the 'original_title'.
- 11 rows with df_movies['vote_average'] = 0
- 11 rows with df_movies['vote_count'] = 0 

In [196]:
# 1037 rows with 'budget' = 0
    # We can use median
# 28 rows with 'genre' = ''
    # Use special value 'Unknown'
# 351 rows with 'production_companies' = ''
    # Use special value 'Independent'
# 174 rows with 'production_countries' = ''
    # Use special value 'Unknown'
# 35 rows with 'runtime' = 0 AND 2 rows with 'runtime' = NaN
    # Use special value -1
# 1427 rows with revenue = 0
    # Check median and mean value of 'revenue' column, select which one appears to be more appropriate 
# 1 row with popularity = 0 (I think this is safe - unpopular movie?)
    # keep this value
# 63 rows with vote_average = 0.0
    # Leave this value 0, as it is possible these movies have received no votes and thus no average can be made (avg of 0 is 0)
# 62 rows with vote_count = 0
    # Leave this value 0, as it is possible these movies have received no votes on the site


# 261 rows where original title != title (Thinking foreign films?)


print("# Rows where budget=0: " + str(len(df_movies.loc[df_movies['budget']==0])))
print("# Rows where genres='[]': " + str(len(df_movies.loc[df_movies['genres']=='[]'])))
print("# Rows where original_language='': " + str(len(df_movies.loc[df_movies['original_language']==''])))
print("# Rows where original_title='': " + str(len(df_movies.loc[df_movies['original_title']==''])))
print("# Rows where popularity=0.0: " + str(len(df_movies.loc[df_movies['popularity']==0.0])))
print("# Rows where production_companies='[]': " + str(len(df_movies.loc[df_movies['production_companies']=='[]'])))
print("# Rows where production_countries='[]': " + str(len(df_movies.loc[df_movies['production_countries']=='[]'])))
print("# Rows where revenue=0: " + str(len(df_movies.loc[df_movies['revenue']==0])))
print("# Rows where title='': " + str(len(df_movies.loc[df_movies['title']==''])))
print("# Rows where runtime=0: " + str(len(df_movies.loc[df_movies['runtime']==0])))
print("# Rows where title != original_title: " + str(len(df_movies.loc[df_movies['title'] !=df_movies['original_title']])))
print("# Rows where vote_average=0.0: " + str(len(df_movies.loc[df_movies['vote_average']==0.0])))
print("# Rows where vote_count=0: " + str(len(df_movies.loc[df_movies['vote_count']==0])))



# Rows where budget=0: 147
# Rows where genres='[]': 4
# Rows where original_language='': 0
# Rows where original_title='': 0
# Rows where popularity=0.0: 0
# Rows where production_companies='[]': 100
# Rows where production_countries='[]': 47
# Rows where revenue=0: 537
# Rows where title='': 0
# Rows where runtime=0: 3
# Rows where title != original_title: 166
# Rows where vote_average=0.0: 11
# Rows where vote_count=0: 11


### Handling the 'Noisy' data
Below we will be dealing with the noisy data that was identified above.
- For df_movies['budget'], we will use the **(median/mean) value** of the column
- For df_movies['genres'], we will apply a **special value 'No listed genres'**
- For df_movies['production_companies'], we will apply a **special value 'Independent film'**
- For df_movies['production_countries'], we will apply a **special value 'Unknown'**
- For df_movies['revenue'], we will use the **(median/mean) value** of the column
- For df_movies['runtime'], we will apply a **special value -1**
- For df_movie['vote_count'] and df_movies['vote_average'], we will refrain from performing any changes as it is possible a movie has no votes thus no vote average
- This investigation also identified that the proper title column to utilize is the df_movies['title'] column

In [197]:
# Code here to make the above adjustments

## Cleaning attributes containing JSON objects as String
The 'genre', 'production_companies' and 'production_countries' attributes contain a JSON as a string, which contains a desired key-value pair. A function was created to extract the desired value within the string, and return it as a list.

### TODO
Some of the rows contain an empty value, denoted by the empty square brackets []. Must decide on how to handle this.

In [198]:
# Cleaning the genres, production_companies and production_countries columns from json string

def extract_json_key(string, key):
    '''
    Helper function used to translate string to list of dictionaries, and extract the 'name' key-value pair
    (Str, Str) -> Str
    Preconditions: None
    '''
    clean_list = json.loads(string)
    name = [x[key] for x in clean_list]
    return ', '.join(name)

# Calling the above function to extract the respective 'name' key-value 
df_movies['genres'] = df_movies['genres'].map(lambda x: extract_json_key(x, 'name'))
df_movies['production_companies'] = df_movies['production_companies'].map(lambda x: extract_json_key(x, 'name'))
df_movies['production_countries'] = df_movies['production_countries'].map(lambda x: extract_json_key(x, 'name'))


## Cleaning the language attributes
This section is focused on cleaning the two langauge attribtues, 'original_language' and 'spoken_language'. 

'original_language' is originally written in ISO 639-1 code, which is a 2 letter abbreviation of a word. Using a dictionary found below, the word is swapped for it's verbose display.

'spoken_language' is a little more difficult, where a film can have multiple spoken languages. This is stored in a similar JSON string as above which stores both the ISO 639-1 code and the full written word. Unfortunately, the full written word may utilize a unicode character (i.e. 'French' written in French has an accent, which stores a unicode character). The solution to this is utililizing our previously made dictionary and search it using the provided ISO 639-1 code. A function takes the list of JSON objects, extracts the ISO 639-1 code and translates it, then returns.

### TODO:
- Some of the columns are empty (denoted by an empty set of square brackets []), an example of this is column k row 2650. Need to make decision on how to handle this

In [199]:
# Create dictionary mapping 2 letter language abbrv. to full word
language_two_code_dict = {
    'en': 'English',
    'fr': 'French',
    'ja': 'Japanese',
    'zh': 'Chinese',
    'es': 'Spanish',
    'de': 'German',
    'hi': 'Hindi',
    'ru': 'Russian',
    'ko': 'Korean',
    'te': 'Telugu',
    'cn': 'Chinese',
    'it': 'Italian',
    'nl': 'Dutch',
    'ta': 'Tamil',
    'sv': 'Swedish',
    'th': 'Thai',
    'da': 'Danish',
    'xx': 'No Language',
    'hu': 'Hungarian',
    'cs': 'Czech',
    'pt': 'Portuguese',
    'is': 'Icelandic',
    'tr': 'Turkish',
    'nb': 'Norwegian Bokmal',
    'af': 'Afrikaans',
    'pl': 'Polish',
    'he': 'Hebrew',
    'ar': 'Arabic',
    'vi': 'Vietnamese',
    'ky': 'Kyrgyz',
    'id': 'Indonesian',
    'ro': 'Romanian',
    'fa': 'Persian',
    'no': 'Norwegian',
    'sl': 'Slovenian',
    'ps': 'Pashto',
    'el': 'Greek'
}

def language_lookup(word):
    '''
    Helper function that searches a ISO 639-1 code in the dictionary and returns the verbose word
    String -> String
    '''
    if(word in language_two_code_dict):
        return language_two_code_dict.get(word)
    else:
        return 'NEED TO DOUBLE CHECK MISSING'

df_movies['original_language'] = df_movies['original_language'].map(lambda x: language_lookup(x))


def translate_and_clean(string):
    '''
    Helper function that takes string of 2 code abbreviations, translates and returns in a String
    (Str) -> Str
    '''
    extract = extract_json_key(string, 'iso_639_1').split(',')
    for i in range(len(extract)):
        extract[i] = extract[i].strip()
        extract[i] = language_lookup(extract[i])
    
    return ', '.join(extract)

# Apply function on each row within 'spoken_languages' column
df_movies['spoken_languages'] = df_movies['spoken_languages'].map(lambda x: translate_and_clean(x))

# Staging the data
Below is the staging part of our script